## OR4 Coadd Metric Viewer

Displays metrics associated with deepCoadds as a series of tables. Metrics that deviate from the column mean by more than 3 standard deviations are highlighted in red.

Bear in mind highlighting assumes a gaussian-ish distribution, meaning that some outlier values may not be highlighted (and vice versa).

Columns named after a band show the number of exposures of that band that have gone into making the coadd (so, "200" under "g" for tract row "500" means that that 200 g-band exposures have gone into creating all the coadds that cover tract 500). 

In [1]:
day_obs = '20230511'
repo = 'embargo'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871'

In [2]:
import lsst.daf.butler as dafButler

import operator
import numpy as np
import pandas as pd
from scipy.stats import zscore

from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-child {overflow: scroll}</style>"))

In [3]:
instrUrl = 'LATISS'
collectionUrl = 'runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871'
plotNavUrl = 'https://usdf-rsp-dev.slac.stanford.edu/plot-navigator'

In [4]:
urlDict = {'g_highSNStars_median':'', 'g_highSNStars_sigmaMad':'', 'g_highSNStars_count':''}
threshDict = {'numBlends':50, 'meanBlendIterations':25, 'meanBlendLogL':-1e5}
threshTypeDict = {'numBlends':'lt', 'meanBlendIterations':'ge', 'meanBlendLogL':'lt'}

In [5]:
def metricBundletToDataFrame(bundleName, refs):
    data = []
    for ref in refs:
        metrics = butler.get(ref)
        row = {'Tract':ref.dataId['tract']}
        for metric in metrics[bundleName]:
            key = '{} {}'.format(metric.metric_name.metric, metric.quantity.unit.to_string())
            value = metric.quantity.value
            row[key] = value
        data.append(row)
    
    df = pd.DataFrame(data)
    df.set_index('Tract', inplace=True)
    return df

def build_urls(s, names=None):
    # Build URLs if column has an associated Plot Navigator page.
    colName = s.name.split()[0]
    if colName not in urlDict:
        return np.array(['-' if np.isnan(v) else f'{v:.2f}' for v in s.values])
    else:
        urlVal = urlDict[colName]
        return [f'<a href="{plotNavUrl}/tract/{index}/{instrUrl}/{collectionUrl}/{urlDict[colName]}">{val:.2f}</a>' for index, val in zip(s.index.values, s.values)]

def flagCells(s):
    colName = s.name.split()[0]
    if colName not in threshDict:
        return np.array(["" for x in s])
    else:
        mask = getattr(operator, threshTypeDict[colName])(s.values, threshDict[colName])
        return np.where(mask, "background-color: red","")

def displayTable(tractTable, metricTable):
    merged = pd.merge(tractTable, metricTable, on='Tract')
    style_df = merged.apply(flagCells, axis=0)
    urls = merged.apply(build_urls, axis=0)
    flagged = urls.style.apply(lambda x: style_df, axis=None)
    with pd.option_context('display.max_rows', 100, 'display.max_columns', 50):
        display(flagged)

In [6]:
butler = dafButler.Butler(repo, collections=collection)
registry = butler.registry

In [7]:
visits = list(registry.queryDimensionRecords('visit', where='day_obs in ({})'.format(day_obs)))
visitIds = [visit.id for visit in visits]
minId, maxId = np.min(visitIds), np.max(visitIds)

coaddRefs = list(registry.queryDatasets('deepCoadd_calexp', where='visit in ({}..{})'.format(minId, maxId)))
data = []
for coaddRef in coaddRefs:
    data.append({'Tract':coaddRef.dataId['tract'], 'Band':coaddRef.dataId['band']})
tractTable = pd.DataFrame(data)
tractTable = tractTable.groupby(tractTable.columns.tolist(),as_index=False).size()
tractTable = tractTable.pivot(index='Tract', columns='Band', values='size')

## Object Table Core Metrics:

In [8]:
refs = list(registry.queryDatasets('objectTableCore_metrics'))

In [9]:
### Blend Metrics

In [10]:
#BlendLogL: Log likelihood of the entire blend in scarlet_lite.
#bundleName = 'blendMetrics'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### E1Diff Metrics

In [11]:
# Ellipticity Residuals (e1 - e1_psf) per tract per band
# highSNStars are SNR > 2700, lowSNStars are SNR > 500
# _count is number of stars in each SNR bin. 
bundleName = 'e1Diff'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct
Tract,,,,,,,,,,,,,,,,,,,,,
5614,324.00,326.00,311.00,0.00,0.01,5.00,0.00,0.01,54.00,-0.01,0.01,4.00,-0.00,0.01,66.00,-0.00,0.01,6.00,0.00,0.01,67.00
5615,361.00,369.00,362.00,-0.00,0.00,9.00,0.00,0.00,81.00,0.00,0.00,13.00,0.00,0.00,115.00,-0.00,0.01,16.00,0.00,0.01,133.00


### E2 Diff Metrics

In [12]:
bundleName = 'e2Diff'
etricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct
Tract,,,,,,,,,,,,,,,,,,,,,
5614,324.00,326.00,311.00,0.00,0.01,5.00,0.00,0.01,54.00,-0.01,0.01,4.00,-0.00,0.01,66.00,-0.00,0.01,6.00,0.00,0.01,67.00
5615,361.00,369.00,362.00,-0.00,0.00,9.00,0.00,0.00,81.00,0.00,0.00,13.00,0.00,0.00,115.00,-0.00,0.01,16.00,0.00,0.01,133.00


In [13]:
### Isolated Deblender Metrics

In [14]:
#bundleName = 'isolatedDeblenderMetrics'
#etricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### Parent Deblender Metrics

In [15]:
#bundleName = 'parentDeblenderMetrics'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### PSF CModel Scatter Metrics

In [16]:
#bundleName = 'psfCModelScatter'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### Shape Size Fractional Diff Metrics

In [17]:
bundleName = 'shapeSizeFractionalDiff'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,g_highSNStars_median pix,g_highSNStars_sigmaMad pix,g_highSNStars_count ct,g_lowSNStars_median pix,g_lowSNStars_sigmaMad pix,g_lowSNStars_count ct,r_highSNStars_median pix,r_highSNStars_sigmaMad pix,r_highSNStars_count ct,r_lowSNStars_median pix,r_lowSNStars_sigmaMad pix,r_lowSNStars_count ct,i_highSNStars_median pix,i_highSNStars_sigmaMad pix,i_highSNStars_count ct,i_lowSNStars_median pix,i_lowSNStars_sigmaMad pix,i_lowSNStars_count ct
Tract,,,,,,,,,,,,,,,,,,,,,
5614,324.00,326.00,311.00,0.00,0.01,5.00,0.00,0.01,54.00,-0.00,0.00,4.00,0.00,0.01,66.00,0.00,0.01,6.00,-0.00,0.01,67.00
5615,361.00,369.00,362.00,-0.00,0.00,9.00,-0.00,0.01,81.00,-0.00,0.00,13.00,-0.00,0.01,115.00,-0.01,0.01,16.00,-0.00,0.01,133.00


### Skipped Deblender Metrics

In [18]:
#bundleName = 'skippedDeblenderMetrics'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)

### Sky Flux Statistic Metrics

In [19]:
bundleName = 'skyFluxStatisticMetric'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,g_medianSky nJy,g_meanSky nJy,g_stdevSky nJy,g_sigmaMADSky nJy,r_medianSky nJy,r_meanSky nJy,r_stdevSky nJy,r_sigmaMADSky nJy,i_medianSky nJy,i_meanSky nJy,i_stdevSky nJy,i_sigmaMADSky nJy
Tract,,,,,,,,,,,,,,,
5614,324.00,326.00,311.00,-12.57,-46.04,1514.20,1085.16,-57.43,11.67,2359.18,1626.97,98.26,83.23,2521.15,1767.40
5615,361.00,369.00,362.00,-57.97,-78.45,1278.74,686.46,80.64,-151.53,3998.42,933.20,3.08,-71.68,2681.28,970.33


### wPerp Cmodel Metrics

In [20]:
bundleName = 'wPerpCModel'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,wPerp_cModelFlux_sigmaMAD mmag,wPerp_cModelFlux_median mmag
Tract,,,,,
5614,324.00,326.00,311.00,32.19,-3.23
5615,361.00,369.00,362.00,23.24,0.04


### wPerp PSFP Metrics

In [21]:
bundleName = 'wPerpPSFP'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,wPerp_psfFlux_sigmaMAD mmag,wPerp_psfFlux_median mmag
Tract,,,,,
5614,324.00,326.00,311.00,35.38,0.78
5615,361.00,369.00,362.00,20.57,-0.61


### xPerpCModel Metrics

In [22]:
bundleName = 'xPerpCModel'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,xPerp_cModelFlux_sigmaMAD mmag,xPerp_cModelFlux_median mmag
Tract,,,,,
5614,324.00,326.00,311.00,-,-
5615,361.00,369.00,362.00,-,-


### xPerpPSFP Metrics

In [23]:
bundleName = 'xPerpPSFP'
metricTable = metricBundletToDataFrame(bundleName, refs)
displayTable(tractTable, metricTable)

,g,i,r,xPerp_psfFlux_sigmaMAD mmag,xPerp_psfFlux_median mmag
Tract,,,,,
5614,324.00,326.00,311.00,-,-
5615,361.00,369.00,362.00,-,-


## Object Table Extended Metrics

In [24]:
#refs = list(registry.queryDatasets('objectTableExtended_metrics'))

### Ap12PSFSky Metrics

In [25]:
#bundleName = 'ap12PsfSky'
#metricTable = metricBundletToDataFrame(bundleName, refs)
#displayTable(tractTable, metricTable)